In [8]:
# general imports
import os #to interact to the file system
import numpy as np #Statistics
import pandas as pd #Database Technology <-> Data preproc & Data Analysis
from matplotlib import pyplot as plt #Visualization
from matplotlib.colors import ListedColormap
import seaborn as sns #Visualization
import missingno as msno
import random 

# from our documents
import OurFunctions as of
import suitable_dataset_finder as sdf

# from Scikit Learn library
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve, roc_auc_score, accuracy_score, make_scorer, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.preprocessing import StandardScaler, OneHotEncoder, RobustScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectKBest, f_classif, RFE
from sklearn.decomposition import PCA
from sklearn.feature_selection import SequentialFeatureSelector as SFS
from sklearn.neighbors import LocalOutlierFactor
from sklearn.tree import plot_tree
from sklearn.calibration import CalibratedClassifierCV

# from Imb Learn
from imblearn.over_sampling import SMOTENC, SMOTE

In [9]:
ASD_phenotypic_original = pd.read_csv(os.path.join('DataSets','Phenotypic Datasets','ASD_phenotypic.csv'))

# DATA CLEANING

In [10]:
# Store them in a new dataset called ASD_clinical
#ASD_clinical = ASD_phenotypic_original[['DX_GROUP']]

In [11]:
#ASD_phenotypic = ASD_phenotypic_original.drop(columns=['DX_GROUP', 'DSM_IV_TR','EYE_STATUS_AT_SCAN', 'SUB_ID'])
ASD_phenotypic = ASD_phenotypic_original.drop(columns=['DSM_IV_TR','EYE_STATUS_AT_SCAN', 'SUB_ID'])

In [12]:
for column in ASD_phenotypic:
    # Replace -9999 and "-9999" with NaN
    ASD_phenotypic[column] = ASD_phenotypic[column].replace(['-9999', -9999], np.NaN)

In [13]:
# Create a mask for rows to keep
filter = (ASD_phenotypic['ADI_R_RSRCH_RELIABLE'] != 0) | (ASD_phenotypic['ADOS_RSRCH_RELIABLE'] != 0)

# Apply the mask to both DataFrames
ASD_phenotypic= ASD_phenotypic[filter]

ASD_phenotypic = ASD_phenotypic.drop(columns=['ADI_R_RSRCH_RELIABLE','ADOS_RSRCH_RELIABLE'])

## Removing excess of missing values

In [14]:

# Features chiave da mantenere
key_features = ['FIQ', 'VIQ', 'PIQ', 'ADI_R_VERBAL_TOTAL_BV', 'ADOS_TOTAL']

def remove_high_missing(df, key_features, balance_column, min_subjects=200, max_missing_percentage=10):
    current_df = df.copy()
    
    # Calcolare la proporzione iniziale del bilanciamento
    initial_balance = current_df[balance_column].value_counts(normalize=True)
    
    while (of.calculate_missing_percentage(current_df) > max_missing_percentage): 
        # Calcola la percentuale di valori mancanti per ciascuna colonna e riga
        missing_percent_features = current_df.isna().mean() * 100
        missing_percent_subjects = current_df.isna().mean(axis=1) * 100
        
        # Filtra le features chiave per non rimuoverle
        non_key_features = missing_percent_features.drop(labels=key_features, errors='ignore')
        
        # Trova la feature o il soggetto con il più alto tasso di missing values
        max_feature_missing = non_key_features.max()
        max_subject_missing = missing_percent_subjects.max()
        
        # Rimuovi la feature o il soggetto con il tasso di missing values più alto
        if max_feature_missing >= max_subject_missing and not non_key_features.empty:
            feature_to_drop = non_key_features.idxmax()
            current_df = current_df.drop(columns=[feature_to_drop])
            print(f"Rimosso feature: {feature_to_drop}")
        elif not missing_percent_subjects.empty:
            scelto_soggetto = False
            while not scelto_soggetto:
                subject_to_drop = missing_percent_subjects.idxmax()
                temp_df = current_df.drop(index=[subject_to_drop])
            
             # Verifica il bilanciamento dopo la rimozione
                current_balance = temp_df[balance_column].value_counts(normalize=True)
                if all(abs(initial_balance - current_balance) <= 0.2):  # Assicurarsi che il bilanciamento non cambi di più del 20%
                    current_df = temp_df
                    scelto_soggetto = True
                    print(f"Rimosso soggetto: {subject_to_drop}")
                else:
                    id = missing_percent_subjects.drop(subject_to_drop)
                    print(f"Soggetto non rimosso per mantenere il bilanciamento: {subject_to_drop}")
            

        # Controllo dello stato attuale del DataFrame
        print(f"Percentuale attuale di missing values: {of.calculate_missing_percentage(current_df):.2f}%")
        if current_df.shape[0] > min_subjects:
            print(f"Numero di soggetti rimanenti: {current_df.shape[0]}")
    
    return current_df


# Applica la funzione di pulizia sul DataFrame
ASD_phenotypic_cleaned = remove_high_missing(ASD_phenotypic, key_features, balance_column='DX_GROUP', min_subjects=200, max_missing_percentage=10)




Rimosso feature: AQ_TOTAL
Percentuale attuale di missing values: 73.00%
Numero di soggetti rimanenti: 1086
Rimosso feature: WISC_IV_VCI
Percentuale attuale di missing values: 72.67%
Numero di soggetti rimanenti: 1086
Rimosso feature: WISC_IV_PRI
Percentuale attuale di missing values: 72.33%
Numero di soggetti rimanenti: 1086
Rimosso feature: WISC_IV_WMI
Percentuale attuale di missing values: 71.99%
Numero di soggetti rimanenti: 1086
Rimosso feature: WISC_IV_PSI
Percentuale attuale di missing values: 71.63%
Numero di soggetti rimanenti: 1086
Rimosso feature: WISC_IV_SIM_SCALED
Percentuale attuale di missing values: 71.26%
Numero di soggetti rimanenti: 1086
Rimosso feature: WISC_IV_VOCAB_SCALED
Percentuale attuale di missing values: 70.88%
Numero di soggetti rimanenti: 1086
Rimosso feature: WISC_IV_INFO_SCALED
Percentuale attuale di missing values: 70.48%
Numero di soggetti rimanenti: 1086
Rimosso feature: WISC_IV_BLK_DSN_SCALED
Percentuale attuale di missing values: 70.07%
Numero di sog

In [15]:
# Store them in a new dataset called ASD_clinical
ASD_clinical = ASD_phenotypic_cleaned[['DX_GROUP']]
# Drop  columns DX_GROUP e fai storage a parte
ASD_phenotypic = ASD_phenotypic_cleaned.drop(columns=['DX_GROUP'])

# DATA PREPROCESSING

In [16]:
ASD_phenotypic = of.inpute_missing_values (ASD_phenotypic, ASD_phenotypic_original)

# CLASSIFICATION

In [17]:
ASD_phenotypic = ASD_phenotypic.drop(columns=["SITE_ID"])
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(ASD_phenotypic, ASD_clinical['DX_GROUP'], test_size=0.3, random_state=42)

In [19]:
categorical_columns = X_train.select_dtypes(include=['object', 'category']).columns
categorical_features = categorical_columns.tolist()

# Inizializza l'oggetto SMOTE-NC specificando gli indici delle colonne categoriche
sampler = SMOTENC(categorical_features=categorical_features, random_state=42)

# Applica SMOTE-NC per generare nuovi esempi sintetici
X_SMOTE, Y_SMOTE = sampler.fit_resample(X_train, y_train)

In [35]:
def general_pipeline(dataset, target, classifier, encoder = True, scaler = True, parameters_grid_search = None, cv = None, feature_selector=False, parameters_feature_selector = None):

    # Definizione delle metriche da utilizzare come scoring
    scoring = {
        'accuracy': make_scorer(accuracy_score),
        'precision': make_scorer(precision_score),
        'recall': make_scorer(recall_score),
        'f1_score': make_scorer(f1_score)
    }

    # Preprocess to make on the train data may include
    # - normalization of the numerical columns
    # - one hot encoding on the categorical columns
    
    categorical_columns = dataset.select_dtypes(include=['object', 'category']).columns

    if not encoder:
        transformers = [('num', RobustScaler(), ~dataset.columns.isin(categorical_columns))]
    elif not scaler:
        transformers = [('cat', OneHotEncoder(handle_unknown='ignore'), categorical_columns)]
    elif not encoder and not scaler:
        transformers = []
    else:
        transformers=[
            ('num', RobustScaler(), ~dataset.columns.isin(categorical_columns)),
            ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_columns),
        ]

    preprocessor = ColumnTransformer(transformers=transformers)

    if isinstance(classifier, KNeighborsClassifier):
        parameter_type = 'classifier__n_neighbors'
        if parameters_grid_search:
            parameters = {parameter_type: parameters_grid_search}  # Valori di n_neighbors da esplorare
        else:
            parameters = {parameter_type: [3, 5, 7, 9, 11]}  # Valori di n_neighbors da esplorare

        feature_selector_type = SFS(classifier)
        selector_parameter = 'feature_selection__n_features_to_select'
        # per default is forward, but if you need to change --> parameters.update({'sfs__direction': ['forward']})
        #feature_selector_type = SelectKBest(score_func=f_classif)
        #selector_parameter = 'feature_selection__k'
        

    if isinstance(classifier, RandomForestClassifier):
        parameter_type = 'classifier__n_estimators'
        if parameters_grid_search:
            parameters = {parameter_type: parameters_grid_search}  
        else:
            parameters = {parameter_type: [10, 50, 200, 500, 1000]} 

        feature_selector_type = RFE(estimator=classifier)
        selector_parameter = 'feature_selection__n_features_to_select'

    if isinstance(classifier, CalibratedClassifierCV):
        parameter_type = 'classifier__estimator__C'
        if parameters_grid_search:
            parameters = {parameter_type: parameters_grid_search}  
        else:
            parameters = {parameter_type: [1]}  

        #feature_selector_type = SelectFromModel(LinearSVC(penalty='l1', dual=False))
        #selector_parameter = 'feature_selection__max_features'
        feature_selector_type = RFE(estimator=LinearSVC(dual=False))
        selector_parameter = 'feature_selection__n_features_to_select'

    if feature_selector:
        # Initialize RFE with the fitted classifier

        pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('feature_selection', feature_selector_type),
        ('classifier', classifier)  
        ])
        if parameters_feature_selector:
            parameters.update({selector_parameter: parameters_feature_selector})
        else: 
            parameters.update({selector_parameter: list(range(1, 23+1))})
        
    else:
    # We define the pipeline
        pipeline = Pipeline([
            ('preprocessor', preprocessor),
            ('classifier', classifier),
            ])

    if cv:
        k_folds = cv
    else:
        k_folds = 5

    # Creazione dell'oggetto GridSearchCV
    grid_search = GridSearchCV(pipeline, parameters, cv= k_folds, scoring=scoring, refit='accuracy')

    # Esecuzione della ricerca a griglia
    grid_search.fit(dataset, target)

    best_model = grid_search.best_estimator_
    best_parameter = grid_search.best_params_[parameter_type]
    best_accuracy = grid_search.best_score_
    print("\n More accurate parameter:")
    print(best_parameter)
    print("Accuration with best parameter:", best_accuracy)

    if feature_selector:
        # Get the support mask and selected feature names from the preprocessed data
        selected_mask = best_model.named_steps['feature_selection'].get_support()
        feature_names = best_model.named_steps['preprocessor'].get_feature_names_out()
        selected_features = feature_names[selected_mask]
        
        print("\nSelected features are:")
        print(f"Number of selected features: {len(selected_features)}")
        print("Selected features:", selected_features)

        non_selected_mask = ~selected_mask
        non_selected_features = feature_names[non_selected_mask]

        print("\nNon selected features are:")
        print(f"Number of non selected features: {len(non_selected_features)}")
        print("Selected features:", non_selected_features)

        return best_model, best_parameter, best_accuracy, selected_features
        
    return best_model, best_parameter, best_accuracy, None
    

In [24]:
classifier = RandomForestClassifier(random_state=1234)

In [26]:
rf_model, rf_parameter, rf_accuracy, _ = general_pipeline(X_SMOTE, Y_SMOTE, classifier)


 More accurate parameter:
50
Accuration with best parameter: 0.9414823008849558


In [27]:
# Valutazione del modello sui dati di test
y_pred = rf_model.predict(X_test)

# Calcolo delle metriche sui dati di test
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("\nValutazione del modello sui dati di test:")
print(f"Accuratezza: {accuracy:.3f}")
print(f"Precisione: {precision:.3f}")
print(f"Richiamo: {recall:.3f}")
print(f"F1-score: {f1:.3f}")



Valutazione del modello sui dati di test:
Accuratezza: 0.970
Precisione: 0.960
Richiamo: 0.992
F1-score: 0.976


## Feature Selection

In [28]:
fs_RF, fs_RF_parameter, fs_RF_accuracy, fs_RF_selected_features = general_pipeline(X_SMOTE, Y_SMOTE, classifier, parameters_grid_search = [50], feature_selector=True)


 More accurate parameter:
50
Accuration with best parameter: 0.9485777496839445

Selected features are:
Number of selected features: 16
Selected features: ['num__AGE_AT_SCAN' 'num__SEX' 'num__FIQ' 'num__VIQ' 'num__PIQ'
 'num__ADI_R_VERBAL_TOTAL_BV' 'num__ADOS_TOTAL' 'cat__FIQ_TEST_TYPE_DAS'
 'cat__FIQ_TEST_TYPE_WASI' 'cat__FIQ_TEST_TYPE_WISC'
 'cat__VIQ_TEST_TYPE_PPVT' 'cat__VIQ_TEST_TYPE_WASI'
 'cat__VIQ_TEST_TYPE_WISC' 'cat__PIQ_TEST_TYPE_Ravens'
 'cat__PIQ_TEST_TYPE_WASI' 'cat__PIQ_TEST_TYPE_WISC']

Non selected features are:
Number of non selected features: 10
Selected features: ['cat__FIQ_TEST_TYPE_WAIS' 'cat__VIQ_TEST_TYPE_DAS'
 'cat__VIQ_TEST_TYPE_Stanford' 'cat__VIQ_TEST_TYPE_WAIS'
 'cat__VIQ_TEST_TYPE_ppvt' 'cat__PIQ_TEST_TYPE_DAS'
 'cat__PIQ_TEST_TYPE_Ravens  ' 'cat__PIQ_TEST_TYPE_Stanford'
 'cat__PIQ_TEST_TYPE_WAIS' 'cat__PIQ_TEST_TYPE_ravens']


In [30]:
# Valutazione del modello sui dati di test
y_pred = fs_RF.predict(X_test)

# Calcolo delle metriche sui dati di test
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("\nValutazione del modello sui dati di test:")
print(f"Accuratezza: {accuracy:.3f}")
print(f"Precisione: {precision:.3f}")
print(f"Richiamo: {recall:.3f}")
print(f"F1-score: {f1:.3f}")



Valutazione del modello sui dati di test:
Accuratezza: 0.965
Precisione: 0.952
Richiamo: 0.992
F1-score: 0.972


## Outlier detection

In [31]:
def outlier_detector(dataset, diagnosis, contamination_factor):
    dataset_outliers = dataset.select_dtypes(include=[np.number])
    X = dataset_outliers.values

    # Initialize Local Outlier Factor
    lof = LocalOutlierFactor(n_neighbors=10, contamination=contamination_factor)  # Adjust parameters as needed
    '''n_neighbors | if too small: model sensible to noise and random outliers
                 if too large: diculties in local outliers detection, in particular if in absence oa a uniform distribution
       contamination | data portion expected as outliers'''
    # Fit the model and predict outliers
    outliers = lof.fit_predict(X)

    # Print number of detected outliers
    print(f"Number of outliers detected: {np.sum(outliers == -1)}")

    # outliers == -1 indicates outliers, 1 indicates inliers
    dataset_outliers['outlier'] = outliers

    outlier_subjects = dataset_outliers[dataset_outliers['outlier'] == -1]

    pd.set_option('display.max_columns', None); outlier_subjects.T

    dataset_without_outliers = dataset[dataset_outliers['outlier'] == 1]
         
    diagnosis_without_outliers = diagnosis[dataset_outliers['outlier'] == 1]

    return dataset_without_outliers, diagnosis_without_outliers

In [36]:
ASD_phenotypic_without_outliers, ASD_diagnosis_without_outliers = outlier_detector(ASD_phenotypic, ASD_clinical, 0.008)

X_train, X_test, y_train, y_test = train_test_split(ASD_phenotypic_without_outliers, ASD_diagnosis_without_outliers['DX_GROUP'], test_size=0.3, random_state=42)

categorical_columns = X_train.select_dtypes(include=['object','category']).columns
categorical_features = categorical_columns.tolist()

# Inizializza l'oggetto SMOTE-NC specificando gli indici delle colonne categoriche
sampler = SMOTENC(categorical_features=categorical_features, random_state=42)

# Applica SMOTE-NC per generare nuovi esempi sintetici
X_SMOTE, Y_SMOTE = sampler.fit_resample(X_train, y_train)


Number of outliers detected: 6


In [39]:
od_RF, od_RF_parameter, od_RF_accuracy, od_RF_selected_features = general_pipeline(X_SMOTE, Y_SMOTE, classifier, parameters_grid_search = [50], feature_selector=True, parameters_feature_selector=[16])


 More accurate parameter:
50
Accuration with best parameter: 0.9434361968306921

Selected features are:
Number of selected features: 16
Selected features: ['num__AGE_AT_SCAN' 'num__SEX' 'num__FIQ' 'num__VIQ' 'num__PIQ'
 'num__ADI_R_VERBAL_TOTAL_BV' 'num__ADOS_TOTAL' 'cat__FIQ_TEST_TYPE_DAS'
 'cat__FIQ_TEST_TYPE_WASI' 'cat__FIQ_TEST_TYPE_WISC'
 'cat__VIQ_TEST_TYPE_DAS' 'cat__VIQ_TEST_TYPE_PPVT'
 'cat__VIQ_TEST_TYPE_WASI' 'cat__VIQ_TEST_TYPE_WISC'
 'cat__PIQ_TEST_TYPE_Ravens' 'cat__PIQ_TEST_TYPE_WASI']

Non selected features are:
Number of non selected features: 8
Selected features: ['cat__FIQ_TEST_TYPE_WAIS' 'cat__VIQ_TEST_TYPE_WAIS'
 'cat__VIQ_TEST_TYPE_ppvt' 'cat__PIQ_TEST_TYPE_DAS'
 'cat__PIQ_TEST_TYPE_Ravens  ' 'cat__PIQ_TEST_TYPE_WAIS'
 'cat__PIQ_TEST_TYPE_WISC' 'cat__PIQ_TEST_TYPE_ravens']


In [40]:
# Valutazione del modello sui dati di test
y_pred = od_RF.predict(X_test)

# Calcolo delle metriche sui dati di test
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("\nValutazione del modello sui dati di test:")
print(f"Accuratezza: {accuracy:.3f}")
print(f"Precisione: {precision:.3f}")
print(f"Richiamo: {recall:.3f}")
print(f"F1-score: {f1:.3f}")



Valutazione del modello sui dati di test:
Accuratezza: 0.950
Precisione: 0.945
Richiamo: 0.976
F1-score: 0.960
